In [18]:
!pip install -U ultralytics

  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-fft 1.3.8 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
datase

In [19]:
!pip uninstall -y numpy
!pip install numpy==1.26.4

Found existing installation: numpy 2.2.6
Uninstalling numpy-2.2.6:
  Successfully uninstalled numpy-2.2.6
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires requests==2.32.3, but you h

In [20]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

print("NumPy version:", np.__version__)
print("All imports successful ✅")

NumPy version: 1.26.4
All imports successful ✅


In [22]:
import os
import random
import shutil
from pathlib import Path
import yaml
import cv2
import matplotlib.pyplot as plt

from ultralytics import YOLO


In [23]:
RAW_DATASET_PATH = "/kaggle/input/redone/RedoneDataset"

In [24]:
WORK_DIR = "/kaggle/working/data"

In [25]:
for split in ["train", "val"]:
    os.makedirs(f"{WORK_DIR}/{split}/images", exist_ok=True)
    os.makedirs(f"{WORK_DIR}/{split}/labels", exist_ok=True)

print("✅ Folder structure created")

✅ Folder structure created


In [26]:
import os

path = "/kaggle/input/redone/RedoneDataset/images/images"

files = [f for f in os.listdir(path)]
print(files)


['1 (607).jpg', '1 (576).jpg', '1 (789).jpg', '1 (550).jpg', '1 (509).jpg', '1 (392).jpg', '1 (521).jpg', '1 (657).jpg', '1 (603).jpg', '1 (445).jpg', '1 (209).jpg', '1 (258).jpg', '1 (663).jpg', '1 (767).jpg', '1 (391).jpg', '1 (103).jpg', '1 (210).jpg', '1 (523).jpg', '1 (144).jpg', '1 (764).jpg', '1 (798).jpg', '1 (60).jpg', '1 (343).jpg', '1 (18).jpg', '1 (4).jpg', '1 (470).jpg', '1 (348).jpg', '1 (422).jpg', '1 (640).jpg', '1 (356).jpg', '1 (444).jpg', '1 (363).jpg', '1 (696).jpg', '1 (567).jpg', '1 (165).jpg', '1 (426).jpg', '1 (667).jpg', '1 (366).jpg', '1 (197).jpg', '1 (519).jpg', '1 (227).jpg', '1 (562).jpg', '1 (586).jpg', '1 (456).jpg', '1 (560).jpg', '1 (41).jpg', '1 (469).jpg', '1 (155).jpg', '1 (44).jpg', '1 (308).jpg', '1 (796).jpg', '1 (680).jpg', '1 (624).jpg', '1 (212).jpg', '1 (49).jpg', '1 (121).jpg', '1 (678).jpg', '1 (670).jpg', '1 (537).jpg', '1 (423).jpg', '1 (281).jpg', '1 (632).jpg', '1 (270).jpg', '1 (485).jpg', '1 (38).jpg', '1 (611).jpg', '1 (156).jpg', '1

In [27]:
import os
import shutil
import random

# ===== CONFIG =====
IMG_DIR = "/kaggle/input/redone/RedoneDataset/images/images"
LBL_DIR = "/kaggle/input/redone/RedoneDataset/labels/labels total"
OUT_DIR = "/kaggle/working/data"
SPLIT_RATIO = 0.8
IMG_EXTS = (".jpg", ".jpeg", ".png")

# ==================
random.seed(42)

# Make output directories
for split in ["train", "val"]:
    os.makedirs(f"{OUT_DIR}/{split}/images", exist_ok=True)
    os.makedirs(f"{OUT_DIR}/{split}/labels", exist_ok=True)

# Get all images
images = [f for f in os.listdir(IMG_DIR) if f.lower().endswith(IMG_EXTS)]
random.shuffle(images)

# Split
split_idx = int(len(images) * SPLIT_RATIO)
train_imgs = images[:split_idx]
val_imgs = images[split_idx:]

# ================== COPY FUNCTION ==================
def copy_files(file_list, split):
    missing = []

    for img in file_list:
        img_path = os.path.join(IMG_DIR, img)
        lbl_name = os.path.splitext(img)[0] + ".txt"
        lbl_path = os.path.join(LBL_DIR, lbl_name)

        # Copy image
        shutil.copy(img_path, f"{OUT_DIR}/{split}/images/{img}")

        # Copy label if exists
        if os.path.exists(lbl_path):
            shutil.copy(lbl_path, f"{OUT_DIR}/{split}/labels/{lbl_name}")
        else:
            missing.append(img)

    print(f"✅ {split.upper()} DONE")
    print(f"❌ Missing labels in {split}: {len(missing)}")

    for m in missing:
        print("   ", m)

# ================== RUN ==================
copy_files(train_imgs, "train")
copy_files(val_imgs, "val")


✅ TRAIN DONE
❌ Missing labels in train: 0
✅ VAL DONE
❌ Missing labels in val: 0


In [28]:
import yaml

data_yaml = {
    "path": "/kaggle/working/data",  # base folder containing train/ and val/
    "train": "train/images",
    "val": "val/images",
    "nc": 4,  # number of classes
    "names": ["logo_peel_off", "double_printing_logo", "logo_dust", "logo_displacement"]
}

with open("/kaggle/working/insoles.yaml", "w") as f:
    yaml.dump(data_yaml, f)

print("YAML file created at /kaggle/working/insoles.yaml")


YAML file created at /kaggle/working/insoles.yaml


In [ ]:
from ultralytics import YOLO

model = YOLO("/kaggle/working/yolov8s.pt")

model.train(
    data="/kaggle/working/insoles.yaml",
    epochs=40,          # optimal for your dataset size
    imgsz=640,
    batch=16,
    device=0,
    patience=10,
    workers=2,
    project="runs",
    name="insoles_defect"
)


In [ ]:
from ultralytics import YOLO
import torch
from pathlib import Path

# Load model
model = YOLO("/kaggle/working/runs/insoles_defect/weights/best.pt")

# Paths
img_path = "/kaggle/working/data/val/images/img014_aug_9.jpg"
label_path = "/kaggle/working/data/val/labels/img014_aug_9.txt"

# Run prediction
results = model.predict(img_path, conf=0.25, iou=0.5, save=False)

# Parse GT labels
gt_boxes = []
gt_classes = []

with open(label_path) as f:
    for line in f:
        cls, x, y, w, h = map(float, line.split())
        gt_classes.append(int(cls))
        gt_boxes.append([x, y, w, h])  # normalized YOLO format

# Parse predictions
pred_boxes = results[0].boxes.xywhn.cpu().numpy()
pred_classes = results[0].boxes.cls.cpu().numpy().astype(int)
pred_scores = results[0].boxes.conf.cpu().numpy()

# IoU calculation
def iou(box1, box2):
    # box format: xywh normalized
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    xa1, ya1 = x1 - w1/2, y1 - h1/2
    xa2, ya2 = x1 + w1/2, y1 + h1/2
    xb1, yb1 = x2 - w2/2, y2 - h2/2
    xb2, yb2 = x2 + w2/2, y2 + h2/2

    inter_w = max(0, min(xa2, xb2) - max(xa1, xb1))
    inter_h = max(0, min(ya2, yb2) - max(ya1, yb1))
    inter = inter_w * inter_h

    union = w1*h1 + w2*h2 - inter
    return inter / union if union > 0 else 0

# Match predictions to GT
TP, FP, FN = 0, 0, 0
matched = set()

for pb, pc in zip(pred_boxes, pred_classes):
    found = False
    for i, (gb, gc) in enumerate(zip(gt_boxes, gt_classes)):
        if i in matched or pc != gc:
            continue
        if iou(pb, gb) >= 0.5:
            TP += 1
            matched.add(i)
            found = True
            break
    if not found:
        FP += 1

FN = len(gt_boxes) - len(matched)

# Metrics
precision = TP / (TP + FP + 1e-6)
recall = TP / (TP + FN + 1e-6)
f1 = 2 * precision * recall / (precision + recall + 1e-6)

print(f"📸 Image Metrics")
print(f"TP: {TP}, FP: {FP}, FN: {FN}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1-score: {f1:.3f}")


In [ ]:
!cat /kaggle/working/data/val/labels/img014_aug_9.txt

In [41]:
!yolo task=detect mode=val model=/kaggle/working/runs/insoles_defect/weights/best.pt \
data=/kaggle/working/insoles.yaml plots=True

Ultralytics 8.3.240 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,127,132 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3328.8±1069.0 MB/s, size: 199.0 KB)
val: Scanning /kaggle/working/data/val/labels.cache... 160 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 160/160 3.4Mit/s 0.0ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 10/10 4.3it/s 2.3s.2s
                   all        160        162      0.984      0.971      0.981      0.732
         logo_peel_off         43         43      0.995          1      0.995      0.691
  double_printing_logo         41         41      0.946      0.976      0.976      0.724
             logo_dust         41         42          1      0.963      0.978      0.821
     logo_displacement         35         36      0.995      0.944      0.976       0.69
Speed: 1.8ms preprocess, 7.6ms inference

In [ ]:
metrics = model.val()
print(metrics)

In [ ]:
!cp /kaggle/working/runs/detect/train/weights/best.pt /kaggle/working/best.pt